## Map Mutations to Protein-Protein and Protein-Nucleic Acid Interfaces
Here we find and visualize the mutations that occur around protein-protein and protein-nucleic acid interfaces.

In [1]:
# Disable Numba: temporary workaround for https://github.com/sbl-sdsc/mmtf-pyspark/issues/288
import os
os.environ['NUMBA_DISABLE_JIT'] = "1"

In [2]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from ipywidgets import interact, IntSlider, FloatSlider, SelectMultiple
import py3Dmol
import pandas as pd

//miniconda3/envs/mmtf-genomics/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# Initialize Spark
spark = SparkSession.builder.appName("3-MapToPolymerInteractions").getOrCreate()

In [4]:
# Enable Arrow-based columnar data transfers between Spark and Pandas dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Input parameters

In [5]:
distance_cutoff = 8 # distance cutoff for finding and visualizing interactions
input_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures
output_file_name = 'mutations3d_polymer_inter.csv' # mutations mapped to polymer interactions

## Read mutation file created in the previous step

In [7]:
pd.set_option('display.max_columns', None)  # show all columns
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color
0,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.A,6XCN,27,27,27,"QHD43416.1:p.27A>V(28), QHD43416.1:p.27A>S(20)...","27A>V(28), 27A>S(20), 27A>T(8)",0.357725,#fc9576
1,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.C,6XCN,27,27,27,"QHD43416.1:p.27A>V(28), QHD43416.1:p.27A>S(20)...","27A>V(28), 27A>S(20), 27A>T(8)",0.357725,#fc9576
2,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.E,6XCN,27,27,27,"QHD43416.1:p.27A>V(28), QHD43416.1:p.27A>S(20)...","27A>V(28), 27A>S(20), 27A>T(8)",0.357725,#fc9576
3,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.A,6XCN,28,28,28,QHD43416.1:p.28Y>H(20),28Y>H(20),0.266225,#fcb499
4,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.C,6XCN,28,28,28,QHD43416.1:p.28Y>H(20),28Y>H(20),0.266225,#fcb499


## Create a list of unique PDB Ids

In [8]:
pdb_ids = list(df.structureId.drop_duplicates())
print("PDB Ids:", pdb_ids)

PDB Ids: ['6XCN']


## Find all polymer-polymer interactions
Find groups (residues) that interact with other polymer chains

In [9]:
structures = mmtfReader.download_mmtf_files(pdb_ids)

queryGroupId, queryChainId, and queryGroupNumber specify the residue that interacts with another chain (targetChainId)

In [10]:
interaction_filter = InteractionFilter(distanceCutoff=distance_cutoff)

interactions = InteractionExtractor().get_polymer_interactions(structures, interaction_filter, level='group').toPandas()
interactions.head()

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,6XCN.E,ASP,A,796,TYR,E,707,706,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,6XCN.E,GLY,A,880,ASN,E,703,702,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,6XCN.A,ALA,E,668,MET,A,869,868,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,6XCN.A,LYS,C,41,ARG,A,577,576,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,6XCN.E,ALA,A,879,TYR,E,707,706,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


## Filter mutations by joining with the interaction data

In [11]:
mt = df.merge(interactions, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','targetGroupNumber'], how='inner')
mt.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.A,6XCN,38,38,38,QHD43416.1:p.38Y>C(4),38Y>C(4),0.123197,#fee1d3,GLN,E,563,TYR,A,38,37,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.A,6XCN,38,38,38,QHD43416.1:p.38Y>C(4),38Y>C(4),0.123197,#fee1d3,LEU,E,560,TYR,A,38,37,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.A,6XCN,38,38,38,QHD43416.1:p.38Y>C(4),38Y>C(4),0.123197,#fee1d3,PHE,E,562,TYR,A,38,37,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.C,6XCN,38,38,38,QHD43416.1:p.38Y>C(4),38Y>C(4),0.123197,#fee1d3,GLN,A,563,TYR,C,38,37,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,0.781126,"Spike glycoprotein, C105 Fab Heavy Chain, C105...",Spike glycoprotein,3.66,6XCN.C,6XCN,38,38,38,QHD43416.1:p.38Y>C(4),38Y>C(4),0.123197,#fee1d3,LEU,A,560,TYR,C,38,37,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


## Save Mappings

In [12]:
mt.to_csv(output_file_name, index=False)

In [13]:
chains = mt.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['6XCN.A', '6XCN.C', '6XCN.E']


## View mutations grouped by protein chain

In [14]:
# Setup viewer
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
 
        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
    
        # don't display water molecules
        viewer1.setStyle({'resn': ['HOH','DOD']}, {})
        
        modified_residues = set()
        rows = group.shape[0]
        for j in range(0, rows):
            if group.iloc[j]['scale'] > logFreq:
                res_num = str(group.iloc[j]['pdbPosition'])
                mod_res = {'resi': res_num, 'chain': chain_id} 
                modified_residues.add(res_num)
            
                # style for mutated residue
                col = group.iloc[j]['color']
                c_col = col + 'Carbon'
                viewer1.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
                viewer1.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}}) 
           
                # style for interacting residues
                surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
                viewer1.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
                viewer1.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
                # style for interacting waters
                waters = {'resn': ['HOH','DOD']}
                waters.update(surroundings)
                viewer1.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
                
                if show_short_label or show_long_label:
                    viewer1.addResLabels(surroundings, {'fontSize':font-2})
            
                # text label
                annotation = group.iloc[j]['annotation']
                variationId = group.iloc[j]['variationId']
                
                if show_short_label:
                    label = annotation
                if show_long_label:
                    label = chain_id + "." + str(res_num) + ": " + variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, mod_res)
              
        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        # print header
        resolution = group.iloc[0]['resolution']
        coverage = group.iloc[0]['coverage']
        description = group.iloc[0]['description']
        print(f"PDB Id:{pdb_id}, chain Id:{chain_id}, resolution:{resolution}, sequence coverage:{coverage:.2f}")
        print(description)
                
        return viewer1.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, 
                    size=z_widget, font=f_widget, logFreq=l_widget, i=s_widget)

def view_image1():
    return viewer1.png()

In [15]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [21]:
view_image1()

## View one mutation at a time
Use the slider to view each mutation. Interacting residues within the `distance_cutoff` are rendered as orange sticks.

In [16]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer2
        viewer2 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer2.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
        
        # highlight chain of interest in blue
        viewer2.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.7}})
        
        # non-polymer style
        viewer2.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # interacting residue style
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']     
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer2.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer2.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})   
        
        # don't display water molecules (except below for interactions)
        viewer2.setStyle({'resn': ['HOH','DOD']}, {})
            
        # select residues in interacting chains by distance from mutation site (same chain is excluded)
        surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
        
        # set style for interacting residues
        viewer2.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
        viewer2.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
    
        # set style for interacting waters
        waters = {'resn': ['HOH','DOD']}
        waters.update(surroundings)
        viewer2.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
        
        annotation = df.iloc[i]['annotation']
        variationId = df.iloc[i]['variationId']      
        
        if show_short_label:
            label = annotation
        if show_long_label:
            label = chain_id + "." + str(res_num) + ": " + variationId
        if show_short_label or show_long_label:
            viewer2.addResLabels(surroundings, {'fontSize':font-2})
            viewer2.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, mod_res)
            
        viewer2.zoomTo(surroundings)
        viewer2.center(mod_res)
        
        if show_surface:
             viewer2.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
        #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", variationId)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer2.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(df)-1, description='structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, 
                    size=z_widget, font=f_widget, i=s_widget)

def view_image2():
    return viewer2.png()

In [17]:
mt_unique = mt.drop_duplicates(["structureChainId","variationId"])

In [18]:
view_single_mutation(mt_unique, distance_cutoff);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [19]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-ligand binding sites: [3-MapToLigandInteractions.ipynb](3-MapToLigandInteractions.ipynb)